In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing

import keras
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV, KFold, cross_val_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

import warnings
warnings.filterwarnings('ignore')

In [10]:
concrete = pd.read_excel('Concrete_Data.xlsx')
concrete.head()

,Cement,BF_slag,Fly_ash,Water,Super_plasticizer,Coarse_aggregate,Fine_aggregate,Age,Comp_str
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.986111
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.269535
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.052780
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.296075


In [11]:
concrete.shape

(1030, 9)

In [12]:
train_inputs = concrete.drop("Comp_str", axis = 1).values
train_targets = concrete["Comp_str"].values

In [13]:
train_inputs =  preprocessing.scale(train_inputs)

In [14]:
print(train_inputs.shape)
print(train_targets.shape)

(1030, 8)
(1030,)


In [15]:
n_cols = train_inputs.shape[1]
input_shape = (n_cols, )

# Creates a model given an activation and learning rate
def create_model(learning_rate=0.01, activation='relu'):
  
    # Create an Adam optimizer with the given learning rate
    opt = Adam(lr=learning_rate)
  
    # Create your binary classification model  
    model = Sequential()
    model.add(Dense(128, activation = activation,
                         input_shape = input_shape,
                         activity_regularizer=regularizers.l2(1e-5)))
    model.add(Dropout(0.50))
    model.add(Dense(128, activation = activation, 
                         activity_regularizer=regularizers.l2(1e-5)))
    model.add(Dropout(0.50))
    model.add(Dense(1, activation = activation))

# Compile the model
    model.compile(optimizer = opt,
                  loss = "mean_absolute_error",
                  metrics=['mse', "mape"])
    return model

In [16]:
# Create a KerasClassifier
model = KerasRegressor(build_fn = create_model,
                       verbose = 0)

# Define the parameters to try out
params = {'activation': ["relu", "tanh"],
          'batch_size': [16, 32, 64], 
          'epochs': [50, 100],
          'learning_rate': [0.01, 0.001, 0.0001]}

# Create a randomize search cv object passing in the parameters to try
random_search = RandomizedSearchCV(model,
                                   param_distributions = params,
                                   cv = KFold(10))

random_search_results = random_search.fit(train_inputs, train_targets)

D:\Anaconda\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [20]:
# Print best score and parameters
print("Best Score: ", random_search_results.best_score_, "\nBest Params: ", random_search_results.best_params_)

Best Score:  -6.072875380516052 
Best Params:  {'learning_rate': 0.01, 'epochs': 50, 'batch_size': 32, 'activation': 'relu'}


In [21]:
n_cols = train_inputs.shape[1]
input_shape = (n_cols, )

# Creates a model given an activation and learning rate
def create_model(learning_rate=0.001, activation='relu'):
  
    # Create an Adam optimizer with the given learning rate
    opt = Adam(lr=learning_rate)
  
    # Create your binary classification model  
    model = Sequential()
    model.add(Dense(128, activation = activation,
                         input_shape = input_shape,
                         activity_regularizer=regularizers.l2(1e-5)))
    model.add(Dropout(0.50))
    model.add(Dense(128, activation = activation, 
                         activity_regularizer=regularizers.l2(1e-5)))
    model.add(Dropout(0.50))
    model.add(Dense(1, activation = activation))

# Compile the model
    model.compile(optimizer = opt,
                  loss = "mean_absolute_error",
                  metrics= ['mse', "mape"])
    return model

In [22]:
# Create a KerasClassifier
model = KerasRegressor(build_fn = create_model,
                       epochs = 100, 
                       batch_size = 16,
                       verbose = 0)

# Calculate the accuracy score for each fold
kfolds = cross_val_score(model,
                         train_inputs,
                         train_targets,
                         cv = 10)

# Print the mean accuracy
print('MSE need to be negated:', kfolds.mean())

# Print the accuracy standard deviation
print('With a standard deviation of:', kfolds.std())

The mean accuracy was: -6.204823613166809
With a standard deviation of: 1.711158259520388
